# **Pricing Analysis - MSA Level**

### *Race*

In [2]:
# New Analysis
from ipyfilechooser import FileChooser
from IPython.display import clear_output, display
from ipywidgets import Output

# Create a FileChooser widget
fc = FileChooser()

# Create an output widget
output = Output()


# Define a function that will be called when a file is selected
def on_file_selected(filechooser):
    # Clear the output widget
    output.clear_output(wait=True)

    file_name = filechooser.selected

    # Analysis
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import plotly.express as px
    import matplotlib.pyplot as plt
    import textwrap
    from scipy import stats as st
    from bioinfokit.analys import stat
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    from IPython.display import display, HTML

    # from IPython.display import clear_output
    from plotly.offline import iplot, init_notebook_mode
    import warnings

    warnings.filterwarnings("ignore", category=FutureWarning)
    init_notebook_mode(connected=True)
    from tqdm.notebook import tqdm

    # formatting of the results section
    BOLD = "<b>"
    UNDERLINE = "<u>"
    ITALIC = "<i>"
    END = "</b></u></i>"

    # Read in the CSV files
    all_my_data = pd.read_csv(file_name)

    # MSA/MD Names Data for Merge
    msa_md_data = pd.read_csv("Data Files/!MSA MD Listing 2023.csv")

    # Merge the dataframes on the 'HmdaMsa' and 'MSA_MD_Code' columns
    merged_data = pd.merge(
        all_my_data, msa_md_data, left_on="HmdaMsa", right_on="MSA_MD_Code"
    )

    # Get the unique loan types from the data
    loan_types = all_my_data["LoanType"].unique()

    def apply_filters(stat_sign, mean_diff_range, show_stats, show_plot):
        display(HTML("<br>"))
        # Loop through each loan type and filter the data accordingly
        for loan_type in tqdm(loan_types):
            display(HTML("<br><br>"))
            display(HTML(UNDERLINE + BOLD + f"Loan Type: {loan_type}" + END))
            display(HTML("<br>"))

            # filtering to only originated loans and non-HECM programs (Loan Type: 'Conventional')
            all_da_data = merged_data[merged_data["LoanType"] == loan_type]
            all_data = all_da_data[all_da_data["HmdaActionTaken"] == "Loan Originated"]
            big_all_data = all_data[all_data["Program"].str.contains("HECM") == False]

            # narrowing down the fields for more accurate analysis
            race_all_data = big_all_data[
                [
                    "LoanId",
                    "AIP",
                    "Rate_Spread",
                    "Race",
                    "branchname",
                    "MSA_MD_Code",
                    "MSA_MD_Name",
                ]
            ]

            res = stat()
            # creating race filters for white vs protected race
            race_filters = {
                "Black or African American": race_all_data["Race"].isin(
                    ["White", "Black or African American"]
                ),
                "American Alaska or Indian": race_all_data["Race"].isin(
                    ["White", "American Alaska or Indian"]
                ),
                "Native Hawaiian or Other Pacific Islander": race_all_data["Race"].isin(
                    ["White", "Native Hawaiian or Other Pacific Islander"]
                ),
                "Asian": race_all_data["Race"].isin(["White", "Asian"]),
            }
            purch_filter = big_all_data["Purpose"].isin(["Purchase"])
            refin_filter = big_all_data["Purpose"].isin(["Refinance"])

            # Loop through each race filter and group the filtered data by 'branchname' for 'Conventional' Loan Type
            for race, race_filter in race_filters.items():
                display(HTML("<br>"))
                display(
                    HTML(
                        "&nbsp;&nbsp;&nbsp;" + ITALIC + BOLD + f"{race} Analysis" + END
                    )
                )
                display(HTML("<br>"))

                filtered_data = race_all_data[race_filter]
                grouped_data = filtered_data.groupby("MSA_MD_Name")

                # Loop through each group and perform a t-test
                for MSA_MD_Name, group in grouped_data:
                    # Check if there are exactly two levels in the 'Race' column
                    if len(group["Race"].unique()) == 2:
                        # Check if there are enough observations for each level
                        """if group['Race'].value_counts().min() >= 10:
                        res.ttest(df=group, xfac='Race', res='AIP', evar=False, test_type=2)
                        """

                        # Check if there are enough observations for each level
                        if (
                            group["Race"].value_counts().min() >= 10
                            and len(group) >= 30
                        ):
                            res.ttest(
                                df=group,
                                xfac="Race",
                                res="AIP",
                                evar=False,
                                test_type=2,
                            )

                            # Calculate loan count for each race
                            loan_counts = group["Race"].value_counts()

                            # Extract the Mean Diff from the summary
                            summary = res.summary
                            lines = summary.split("\n")
                            mean_diff_line = lines[4]
                            mean_diff = float(mean_diff_line.split()[-1])

                            # Extract the p-value from the summary
                            p_value_line = lines[8]
                            p_value = float(p_value_line.split()[-1])

                            # Round the p-value to 4 decimal places
                            mean_diff = round(mean_diff, 2)
                            p_value = round(p_value, 4)

                            if p_value <= stat_sign and mean_diff >= mean_diff_range:
                                # Print the MSA_MD_Name, Mean and P Value, and the full Results
                                display(
                                    HTML(
                                        "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                        + BOLD
                                        + MSA_MD_Name
                                        + END
                                    )
                                )
                                display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + f"Group: {race}"))
                                display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + f"Loan Type: {loan_type}"))
                                display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + f"BPS Diff: {BOLD}{mean_diff}{END}"))
                                display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + f"p-value: {p_value}"))
                                display(HTML("<br>"))
                                for race_name, count in loan_counts.items():
                                    display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + f"Loan Count ({race_name}): {count}"))
                                display(HTML("<br>"))

                                if show_stats == "Include":
                                    display(
                                        HTML(
                                            BOLD
                                            + UNDERLINE
                                            + f"Statistical Results"
                                            + END
                                        )
                                    )
                                    display(HTML(f"<pre>{res.summary}</pre>"))
                                    display(HTML("<br>"))

                                # Create a strip chart of the AIP values grouped by Race using plotly.express
                                if show_plot == "Strip Chart ":
                                    display(
                                        HTML(
                                            "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                            + BOLD
                                            + UNDERLINE
                                            + f"Strip Chart"
                                            + END
                                        )
                                    )
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style("darkgrid")
                                    stripplot = sns.stripplot(
                                        x="Race",
                                        y="AIP",
                                        hue="Race",
                                        data=group,
                                        jitter=0.4,
                                        dodge=True,
                                        linewidth=1,
                                        palette="Dark2",
                                        legend=False,
                                        size=8,
                                    )
                                    plt.title(
                                        f"MSA/MD: {MSA_MD_Name} ({loan_type} Loans)"
                                    )
                                    plt.show()
                                    display(HTML("<br>"))

                                # Create a box chart of the AIP values grouped by Sex using plotly.express
                                """if show_plot == 'Boxplot ':
                                    display(HTML(BOLD + UNDERLINE + f'Boxplot' + END))
                                    display(HTML("<br>"))
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style('darkgrid')
                                    sns.boxplot(x='Race', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                    sns.stripplot(x='Race', y='AIP', hue='Race', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=6)
                                    plt.show()
                                    display(HTML("<br>"))"""

                                if show_plot == "Boxplot ":
                                    display(
                                        HTML(
                                            "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                            + BOLD
                                            + UNDERLINE
                                            + f"Boxplot"
                                            + END
                                        )
                                    )
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style("darkgrid")
                                    # Your original color
                                    original_color = (234, 234, 242)
                                    # Convert to RGB color for seaborn
                                    seaborn_color = tuple(
                                        [val / 255 for val in original_color]
                                    ) + (
                                        0,
                                    )  # Add alpha for transparency
                                    ax = sns.boxplot(
                                        x="Race",
                                        y="AIP",
                                        data=group,
                                        color=seaborn_color,
                                        showmeans=True,
                                        fliersize=0,
                                        meanprops={
                                            "marker": "o",
                                            "markerfacecolor": "white",
                                            "markeredgecolor": "black",
                                            "markersize": "10",
                                        },
                                    )
                                    for patch in ax.artists:
                                        patch.set_zorder(1)
                                    strip = sns.stripplot(
                                        x="Race",
                                        y="AIP",
                                        hue="Race",
                                        data=group,
                                        jitter=0.4,
                                        dodge=False,
                                        linewidth=1,
                                        palette="Dark2",
                                        legend=False,
                                        size=8,
                                    )
                                    for collection in strip.collections:
                                        collection.set_zorder(2)
                                    plt.title(f"MSA/MD: {MSA_MD_Name} ({loan_type} Loans)")
                                    plt.show()
                                    display(HTML("<br>"))

                                # Print the list of loans included in the analysis
                                if show_plot == "List of Loans ":
                                    group = group.sort_values("Race", ascending=True)
                                    display(
                                        HTML(
                                            "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                            + BOLD
                                            + UNDERLINE
                                            + f"List of Loans"
                                            + END
                                        )
                                    )
                                    display(HTML("<br>"))
                                    # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                    html_table = group.to_html(index=False)

                                    # Add inline CSS to adjust the font size
                                    html_table = (
                                        "<style> table {font-size: 0.9em;} </style>"
                                        + html_table
                                    )
                                    display(HTML(html_table))
                                    display(HTML("<br>"))

    # Create a radio button widget with options for statistically significant results
    stat_sign_widget = widgets.ToggleButtons(
        options=[0.05, 0.01, 0.2, 1.0],
        description="Statistical Significance:",
        disabled=False,
        button_style="",  # 'success', 'info', 'warning', 'danger' or ''
        tooltips=["95% (Default)", "99%", "80%", "100%"],
    )

    # Create a radio button widget with options for mean diff ranges
    mean_diff_widget = widgets.ToggleButtons(
        options=[0.10, 0.15, 0.20, 0.25],
        description="Mean Diff (BPS):",
        disabled=False,
        button_style="",  # 'success', 'info', 'warning', 'danger' or ''
        tooltips=[],
    )

    # Create a radio button widget with options for showing statistic results
    show_stats_widget = widgets.ToggleButtons(
        options=["Exclude", "Include"],
        description="Show Statistical Results:",
        disabled=False,
        button_style="",  # 'success', 'info', 'warning', 'danger' or ''
        tooltips=["Exclude the stats (Default)", "Include the stats"],
    )

    # Create a radio button widget with options for showing plots
    show_plot_widget = widgets.ToggleButtons(
        options=["None ", "Strip Chart ", "Boxplot ", "List of Loans "],
        description="Charts and List of Loans:",
        disabled=False,
        button_style="",  # 'success', 'info', 'warning', 'danger' or ''
        tooltips=["Exclude the plots (Default)", "Include the plots"],
        icons=["times", "area-chart", "bar-chart", "list"],
    )

    # Use the interact function to apply the selected filters to the code
    interact(
        apply_filters,
        stat_sign=stat_sign_widget,
        mean_diff_range=mean_diff_widget,
        show_stats=show_stats_widget,
        show_plot=show_plot_widget,
    )


# Set the function to be called when a file is selected
fc.register_callback(on_file_selected)

# Display the FileChooser widget
display(fc)
fc.title = "<b>Select File (.csv) for Race MSA Analysis</b>"

# Display the output widget
display(output)

FileChooser(path='C:\Users\colby.kellersberger\Documents\Fair Lending Analysis\Pricing', filename='', title=''…

Output()

---

### *Ethnicity, Age, Sex*

In [ ]:
# New Analysis
from ipyfilechooser import FileChooser
from IPython.display import clear_output, display
from ipywidgets import Output

# Create a FileChooser widget
fc = FileChooser()

# Create an output widget
output = Output()


# Define a function that will be called when a file is selected
def on_file_selected(filechooser):
    # Clear the output widget
    output.clear_output(wait=True)

    file_name = filechooser.selected

    # Analysis
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import warnings

    warnings.filterwarnings("ignore", category=FutureWarning)
    import plotly.express as px
    import matplotlib.pyplot as plt
    from scipy import stats as st
    from bioinfokit.analys import stat
    from ipywidgets import (
        interact,
        interactive,
        fixed,
        interact_manual,
        Output,
        IntProgress,
    )
    import ipywidgets as widgets
    from IPython.display import display, HTML
    from plotly.offline import iplot, init_notebook_mode

    init_notebook_mode(connected=True)
    from tqdm.notebook import tqdm

    # formatting of the results section
    BOLD = "<b>"
    UNDERLINE = "<u>"
    ITALIC = "<i>"
    END = "</b></u></i>"

    # reading in data
    all_my_data = pd.read_csv(file_name)

    # MSA/MD Names Data for Merge
    msa_md_data = pd.read_csv("Data Files/!MSA MD Listing 2023.csv")

    # Merge the dataframes on the 'HmdaMsa' and 'MSA_MD_Code' columns
    merged_data = pd.merge(
        all_my_data, msa_md_data, left_on="HmdaMsa", right_on="MSA_MD_Code"
    )

    # Get the unique loan types from the data
    loan_types = all_my_data["LoanType"].unique()

    def apply_filters(analysis, stat_sign, mean_diff_range, show_stats, show_plot):
        display(HTML("<br>"))
        # Loop through each loan type and filter the data accordingly
        for loan_type in tqdm(loan_types):
            display(HTML("<br>"))
            display(HTML(UNDERLINE + BOLD + f"Loan Type: {loan_type}" + END))
            display(HTML("<br>"))

            # filtering to only originated loans and non-HECM programs (Loan Programs: 'Conventional')
            all_da_data = merged_data[merged_data["LoanType"] == loan_type]
            all_data = all_da_data[all_da_data["HmdaActionTaken"] == "Loan Originated"]
            big_all_data = all_data[all_data["Program"].str.contains("HECM") == False]

            # narrowing down the fields for more accurate analysis
            gender_all_data = big_all_data[
                [
                    "LoanId",
                    "AIP",
                    "Sex",
                    "branchname",
                    "Rate_Spread",
                    "MSA_MD_Code",
                    "MSA_MD_Name",
                ]
            ]
            age_all_data = big_all_data[
                [
                    "LoanId",
                    "AIP",
                    "Age",
                    "branchname",
                    "Rate_Spread",
                    "MSA_MD_Code",
                    "MSA_MD_Name",
                ]
            ]
            ethn_all_data = big_all_data[
                [
                    "LoanId",
                    "AIP",
                    "Class",
                    "branchname",
                    "Rate_Spread",
                    "MSA_MD_Code",
                    "MSA_MD_Name",
                ]
            ]

            res = stat()
            # creating filters
            # Gender Filter
            gender_filter = gender_all_data["Sex"].isin(["Male", "Female"])

            # Filter out rows with 'Exclude' in the 'Class' column
            ethn_all_data = ethn_all_data[ethn_all_data["Class"] != "EXCLUDED"]

            # Create a boolean mask to exclude rows with 'Excluded' in the 'Sex' column
            gender_mask = gender_all_data["Sex"] != "Excluded"

            # Apply the mask to the data frame
            gender_all_data = gender_all_data[gender_mask]

            # applying filters
            gender = gender_all_data[gender_filter]

            # Group the SEX data by 'MSA_MD_Name'
            if analysis == "All" or analysis == "Sex":
                grouped_gender_data = gender.groupby("MSA_MD_Name")
                display(
                    HTML("&nbsp;&nbsp;&nbsp;" + ITALIC + BOLD + f"Sex Analysis" + END)
                )
                display(HTML("<br>"))

                # Loop through each group and perform a t-test
                for MSA_MD_Name, group in grouped_gender_data:
                    # Check if there are enough observations for each level
                    """if group['Sex'].value_counts().min() >= 10:
                    res.ttest(df=group, xfac='Sex', res='AIP', evar=False, test_type=2)
                    """

                    # Check if there are enough observations for each level
                    if group["Sex"].value_counts().min() >= 10 and len(group) >= 30:
                        res.ttest(
                            df=group, xfac="Sex", res="AIP", evar=False, test_type=2
                        )

                        # Calculate loan count by group
                        loan_count_by_group = group["Sex"].value_counts()

                        # Extract the Mean Diff from the summary
                        summary = res.summary
                        lines = summary.split("\n")
                        mean_diff_line = lines[4]
                        mean_diff = float(mean_diff_line.split()[-1])

                        # Extract the p-value from the summary
                        p_value_line = lines[8]
                        p_value = float(p_value_line.split()[-1])

                        # Round the mean_diff and p-value to 2 decimal places
                        mean_diff = round(mean_diff, 2)
                        p_value = round(p_value, 4)

                        if p_value <= stat_sign and mean_diff >= mean_diff_range:
                            # Print the MSA_MD_Name, Mean and P Value, and the full Results
                            display(
                                HTML(
                                    "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                    + BOLD
                                    + MSA_MD_Name
                                    + END
                                )
                            )
                            display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + f"Group: Female"))
                            display(
                                HTML(
                                    "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                    + f"Loan Type: {loan_type}"
                                )
                            )
                            display(
                                HTML(
                                    "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                    + f"BPS Diff: {BOLD}{mean_diff}{END}"
                                )
                            )
                            display(
                                HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + f"p-value: {p_value}")
                            )
                            #display(HTML("<br>"))
                            for group_name, count in loan_count_by_group.items():
                                display(
                                    HTML(
                                        "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                        + f"Loan Count ({group_name}): {count}"
                                    )
                                )
                            display(HTML("<br>"))

                            if show_stats == "Include":
                                display(
                                    HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"Statistical Results" + END))
                                display(HTML(f"<pre>{res.summary}</pre>"))
                                display(HTML("<br>"))

                            # Create a strip chart of the AIP values grouped by Sex using plotly.express
                            if show_plot == "Strip Chart ":
                                display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"Strip Chart" + END))
                                plt.figure(figsize=(10, 6))
                                sns.set_style("darkgrid")
                                stripplot = sns.stripplot(
                                    x="Sex",
                                    y="AIP",
                                    hue="Sex",
                                    data=group,
                                    jitter=0.4,
                                    dodge=True,
                                    linewidth=1,
                                    palette="Dark2",
                                    legend=False,
                                    size=7,
                                )
                                plt.title(f"MSA/MD: {MSA_MD_Name} ({loan_type} Loans)")
                                plt.show()
                                display(HTML("<br>"))

                            # Create a box chart of the AIP values grouped by Sex using plotly.express
                            """if show_plot == "Boxplot ":
                                display(
                                    HTML(BOLD + UNDERLINE + f"Boxplot" + END)
                                )
                                plt.figure(figsize=(10, 6))
                                sns.set_style("darkgrid")
                                sns.boxplot(
                                    x="Sex",
                                    y="AIP",
                                    data=group,
                                    color="skyblue",
                                    showmeans=True,
                                    fliersize=0,
                                    meanprops={
                                        "marker": "o",
                                        "markerfacecolor": "white",
                                        "markeredgecolor": "black",
                                        "markersize": "10",
                                    },
                                )
                                sns.stripplot(
                                    x="Sex",
                                    y="AIP",
                                    hue="Sex",
                                    data=group,
                                    jitter=0.4,
                                    dodge=False,
                                    linewidth=1,
                                    palette="Dark2",
                                    legend=True,
                                    size=7,
                                )
                                plt.show()
                                display(HTML("<br>"))"""

                            if show_plot == "Boxplot ":
                                display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"Boxplot" + END))
                                plt.figure(figsize=(10, 6))
                                sns.set_style("darkgrid")
                                # Your original color
                                original_color = (234, 234, 242)
                                # Convert to RGB color for seaborn
                                seaborn_color = tuple(
                                    [val / 255 for val in original_color]
                                ) + (
                                    0,
                                )  # Add alpha for transparency
                                ax = sns.boxplot(
                                    x="Sex",
                                    y="AIP",
                                    data=group,
                                    color=seaborn_color,
                                    showmeans=True,
                                    fliersize=0,
                                    meanprops={
                                        "marker": "o",
                                        "markerfacecolor": "white",
                                        "markeredgecolor": "black",
                                        "markersize": "10",
                                    },
                                )
                                for patch in ax.artists:
                                    patch.set_zorder(1)
                                strip = sns.stripplot(
                                    x="Sex",
                                    y="AIP",
                                    hue="Sex",
                                    data=group,
                                    jitter=0.4,
                                    dodge=False,
                                    linewidth=1,
                                    palette="Dark2",
                                    legend=False,
                                    size=8,
                                )
                                for collection in strip.collections:
                                    collection.set_zorder(2)
                                plt.title(f"MSA/MD: {MSA_MD_Name} ({loan_type} Loans)")
                                plt.show()
                                display(HTML("<br>"))

                            # Print the list of loans included in the analysis
                            if show_plot == "List of Loans ":
                                group = group.sort_values("Sex", ascending=False)
                                display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"List of Loans" + END))
                                display(HTML("<br>"))
                                # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                html_table = group.to_html(index=False)

                                # Add inline CSS to adjust the font size
                                html_table = (
                                    "<style> table {font-size: 0.9em;} </style>"
                                    + html_table
                                )
                                display(HTML(html_table))
                                display(HTML("<br>"))

            elif analysis == "Age" or analysis == "Ethnicity":
                pass

            # Group the AGE data by 'MSA_MD_Name'
            if analysis == "All" or analysis == "Age":
                grouped_age_data = age_all_data.groupby("MSA_MD_Name")
                display(HTML("<br>"))
                display(
                    HTML("&nbsp;&nbsp;&nbsp;" + ITALIC + BOLD + f"Age Analysis" + END)
                )
                display(HTML("<br>"))

                # Loop through each group and perform a t-test
                for MSA_MD_Name, group in grouped_age_data:
                    # Check if there are exactly two levels in the 'Age' column
                    if len(group["Age"].unique()) != 2:
                        # print('Not exactly two levels in the Age column')
                        continue

                    # Check if there are enough observations for each level
                    """if group['Age'].value_counts().min() >= 10:
                        res.ttest(df=group, xfac='Age', res='AIP', evar=False, test_type=2)"""

                    # Check if there are enough observations for each level
                    if group["Age"].value_counts().min() >= 10 and len(group) >= 30:
                        res.ttest(
                            df=group, xfac="Age", res="AIP", evar=False, test_type=2
                        )

                        # Calculate loan count by group
                        loan_count_by_group = group["Age"].value_counts()

                        # Extract the Mean Diff from the summary
                        summary = res.summary
                        lines = summary.split("\n")
                        mean_diff_line = lines[4]
                        mean_diff = float(mean_diff_line.split()[-1])

                        # Extract the p-value from the summary
                        p_value_line = lines[8]
                        p_value = float(p_value_line.split()[-1])

                        # Round the mean_diff and p-value to 2 decimal places
                        mean_diff = round(mean_diff, 2)
                        p_value = round(p_value, 4)

                        if p_value <= stat_sign and mean_diff >= mean_diff_range:
                            # Print the MSA_MD_Name, Mean and P Value, and the full Results
                            display(
                                HTML(
                                    "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                    + BOLD
                                    + MSA_MD_Name
                                    + END
                                )
                            )
                            display(
                                HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + f"Group: Seniors")
                            )
                            display(
                                HTML(
                                    "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                    + f"Loan Type: {loan_type}"
                                )
                            )
                            display(
                                HTML(
                                    "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                    + f"BPS Diff: {BOLD}{mean_diff}{END}"
                                )
                            )
                            display(
                                HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + f"p-value: {p_value}")
                            )
                            #display(HTML("<br>"))
                            for group_name, count in loan_count_by_group.items():
                                display(
                                    HTML(
                                        "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                        + f"Loan Count ({group_name}): {count}"
                                    )
                                )
                            display(HTML("<br>"))

                            if show_stats == "Include":
                                display(
                                    HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"Statistical Results" + END))
                                display(HTML(f"<pre>{res.summary}</pre>"))
                                display(HTML("<br>"))

                            # Create a strip chart of the AIP values grouped by Age using plotly.express
                            if show_plot == "Strip Chart ":
                                display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"Strip Chart" + END))
                                plt.figure(figsize=(10, 6))
                                sns.set_style("darkgrid")
                                stripplot = sns.stripplot(
                                    x="Age",
                                    y="AIP",
                                    hue="Age",
                                    data=group,
                                    jitter=0.4,
                                    dodge=True,
                                    linewidth=1,
                                    palette="Dark2",
                                    legend=False,
                                    size=7,
                                )
                                plt.title(f"MSA/MD: {MSA_MD_Name} ({loan_type} Loans)")
                                plt.show()
                                display(HTML("<br>"))

                            # Create a box chart of the AIP values grouped by Sex using plotly.express
                            """if show_plot == "Boxplot ":
                                display(
                                    HTML(BOLD + UNDERLINE + f"Boxplot" + END)
                                )
                                display(HTML("<br>"))
                                plt.figure(figsize=(10, 6))
                                sns.set_style("darkgrid")
                                sns.boxplot(
                                    x="Age",
                                    y="AIP",
                                    data=group,
                                    color="skyblue",
                                    showmeans=True,
                                    fliersize=0,
                                    meanprops={
                                        "marker": "o",
                                        "markerfacecolor": "white",
                                        "markeredgecolor": "black",
                                        "markersize": "10",
                                    },
                                )
                                sns.stripplot(
                                    x="Age",
                                    y="AIP",
                                    hue="Age",
                                    data=group,
                                    jitter=0.4,
                                    dodge=False,
                                    linewidth=1,
                                    palette="Dark2",
                                    legend=True,
                                    size=7,
                                )
                                plt.show()
                                display(HTML("<br>"))"""

                            if show_plot == "Boxplot ":
                                display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"Boxplot" + END))
                                plt.figure(figsize=(10, 6))
                                sns.set_style("darkgrid")
                                # Your original color
                                original_color = (234, 234, 242)
                                # Convert to RGB color for seaborn
                                seaborn_color = tuple(
                                    [val / 255 for val in original_color]
                                ) + (
                                    0,
                                )  # Add alpha for transparency
                                ax = sns.boxplot(
                                    x="Age",
                                    y="AIP",
                                    data=group,
                                    color=seaborn_color,
                                    showmeans=True,
                                    fliersize=0,
                                    meanprops={
                                        "marker": "o",
                                        "markerfacecolor": "white",
                                        "markeredgecolor": "black",
                                        "markersize": "10",
                                    },
                                )
                                for patch in ax.artists:
                                    patch.set_zorder(1)
                                strip = sns.stripplot(
                                    x="Age",
                                    y="AIP",
                                    hue="Age",
                                    data=group,
                                    jitter=0.4,
                                    dodge=False,
                                    linewidth=1,
                                    palette="Dark2",
                                    legend=False,
                                    size=8,
                                )
                                for collection in strip.collections:
                                    collection.set_zorder(2)
                                plt.title(f"MSA/MD: {MSA_MD_Name} ({loan_type} Loans)")
                                plt.show()
                                display(HTML("<br>"))

                            # Print the list of loans included in the analysis
                            if show_plot == "List of Loans ":
                                group = group.sort_values("Age", ascending=False)
                                display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"List of Loans" + END))
                                display(HTML("<br>"))

                                # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                html_table = group.to_html(index=False)

                                # Add inline CSS to adjust the font size
                                html_table = (
                                    "<style> table {font-size: 0.9em;} </style>"
                                    + html_table
                                )
                                display(HTML(html_table))
                                display(HTML("<br>"))

            elif analysis == "Sex" or analysis == "Ethnicity":
                pass

            # Group the ETHNICITY data by 'MSA_MD_Name'
            if analysis == "All" or analysis == "Ethnicity":
                grouped_ethn_data = ethn_all_data.groupby("MSA_MD_Name")
                display(HTML("<br>"))
                display(HTML("&nbsp;&nbsp;&nbsp;" + BOLD + ITALIC + f"Ethnicity Analysis" + END))
                display(HTML("<br>"))

                # Loop through each group and perform a t-test
                for MSA_MD_Name, group in grouped_ethn_data:
                    # Check if there are exactly two levels in the 'Class' column
                    if len(group["Class"].unique()) == 2:
                        # Check if there are enough observations for each level
                        """if group['Class'].value_counts().min() >= 10:
                        res.ttest(df=group, xfac='Class', res='AIP', evar=False, test_type=2)
                        """

                        # Check if there are enough observations for each level
                        if (
                            group["Class"].value_counts().min() >= 10
                            and len(group) >= 30
                        ):
                            res.ttest(
                                df=group,
                                xfac="Class",
                                res="AIP",
                                evar=False,
                                test_type=2,
                            )

                            # Calculate loan count by group
                            loan_count_by_group = group["Class"].value_counts()

                            # Extract the Mean Diff from the summary
                            summary = res.summary
                            lines = summary.split("\n")
                            mean_diff_line = lines[4]
                            mean_diff = float(mean_diff_line.split()[-1])

                            # Extract the p-value from the summary
                            p_value_line = lines[8]
                            p_value = float(p_value_line.split()[-1])

                            # Round the mean_diff and p-value to 2 decimal places
                            mean_diff = round(mean_diff, 2)
                            p_value = round(p_value, 4)

                            if p_value <= stat_sign and mean_diff >= mean_diff_range:
                                # Print the MSA_MD_Name, Mean and P Value, and the full Results
                                display(
                                    HTML(
                                        "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                        + BOLD
                                        + MSA_MD_Name
                                        + END
                                    )
                                )
                                display(
                                    HTML(
                                        "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                        + f"Group: Hispanic or Latino"
                                    )
                                )
                                display(
                                    HTML(
                                        "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                        + f"Loan Type: {loan_type}"
                                    )
                                )
                                display(
                                    HTML(
                                        "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                        + f"BPS Diff: {BOLD}{mean_diff}{END}"
                                    )
                                )
                                display(
                                    HTML(
                                        "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                        + f"p-value: {p_value}"
                                    )
                                )
                                #display(HTML("<br>"))
                                for group_name, count in loan_count_by_group.items():
                                    display(
                                        HTML(
                                            "&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
                                            + f"Loan Count ({group_name}): {count}"
                                        )
                                    )
                                display(HTML("<br>"))

                                if show_stats == "Include":
                                    display(
                                        HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"Statistical Results" + END))
                                    display(HTML(f"<pre>{res.summary}</pre>"))
                                    display(HTML("<br>"))

                                # Create a strip chart of the AIP values grouped by Class using plotly.express
                                if show_plot == "Strip Chart ":
                                    display(
                                        HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"Strip Chart" + END)
                                    )
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style("darkgrid")
                                    stripplot = sns.stripplot(
                                        x="Class",
                                        y="AIP",
                                        hue="Class",
                                        data=group,
                                        jitter=0.4,
                                        dodge=True,
                                        linewidth=1,
                                        palette="Dark2",
                                        legend=False,
                                        size=7,
                                    )
                                    plt.title(
                                        f"MSA/MD: {MSA_MD_Name} ({loan_type} Loans)"
                                    )
                                    plt.show()
                                    display(HTML("<br>"))

                                # Create a box chart of the AIP values grouped by Sex using plotly.express
                                """if show_plot == "Boxplot ":
                                    display(
                                        HTML(
                                            BOLD
                                            + UNDERLINE
                                            + f"Loan Pricing Plot"
                                            + END
                                        )
                                    )
                                    display(HTML("<br>"))
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style("darkgrid")
                                    sns.boxplot(
                                        x="Class",
                                        y="AIP",
                                        data=group,
                                        color="skyblue",
                                        showmeans=True,
                                        fliersize=0,
                                        meanprops={
                                            "marker": "o",
                                            "markerfacecolor": "white",
                                            "markeredgecolor": "black",
                                            "markersize": "10",
                                        },
                                    )
                                    sns.stripplot(
                                        x="Class",
                                        y="AIP",
                                        hue="Class",
                                        data=group,
                                        jitter=0.4,
                                        dodge=False,
                                        linewidth=1,
                                        palette="Dark2",
                                        legend=True,
                                        size=7,
                                    )
                                    plt.show()
                                    display(HTML("<br>"))"""

                                if show_plot == "Boxplot ":
                                    display(HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"Boxplot" + END))
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style("darkgrid")
                                    # Your original color
                                    original_color = (234, 234, 242)
                                    # Convert to RGB color for seaborn
                                    seaborn_color = tuple(
                                        [val / 255 for val in original_color]
                                    ) + (
                                        0,
                                    )  # Add alpha for transparency
                                    ax = sns.boxplot(
                                        x="Class",
                                        y="AIP",
                                        data=group,
                                        color=seaborn_color,
                                        showmeans=True,
                                        fliersize=0,
                                        meanprops={
                                            "marker": "o",
                                            "markerfacecolor": "white",
                                            "markeredgecolor": "black",
                                            "markersize": "10",
                                        },
                                    )
                                    for patch in ax.artists:
                                        patch.set_zorder(1)
                                    strip = sns.stripplot(
                                        x="Class",
                                        y="AIP",
                                        hue="Class",
                                        data=group,
                                        jitter=0.4,
                                        dodge=False,
                                        linewidth=1,
                                        palette="Dark2",
                                        legend=False,
                                        size=8,
                                    )
                                    for collection in strip.collections:
                                        collection.set_zorder(2)
                                    plt.title(f"MSA/MD: {MSA_MD_Name} ({loan_type} Loans)")
                                    plt.show()
                                    display(HTML("<br>"))

                                # Print the list of loans included in the analysis
                                if show_plot == "List of Loans ":
                                    group = group.sort_values("Class", ascending=False)
                                    display(
                                        HTML("&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;" + BOLD + UNDERLINE + f"List of Loans" + END)
                                    )
                                    display(HTML("<br>"))

                                    # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                    html_table = group.to_html(index=False)

                                    # Add inline CSS to adjust the font size
                                    html_table = (
                                        "<style> table {font-size: 0.9em;} </style>"
                                        + html_table
                                    )
                                    display(HTML(html_table))
                                    display(HTML("<br>"))

            elif analysis == "Age" or analysis == "Sex":
                pass

    # Create a radio button widget with options for statistically significant results
    analysis_widget = widgets.ToggleButtons(
        options=["All", "Ethnicity", "Age", "Sex"],
        description="Demographic Analysis:",
        disabled=False,
        button_style="",  # 'success', 'info', 'warning', 'danger' or ''
        tooltips=["95% (Default)", "99%", "80%", "100%"],
    )

    # Create a radio button widget with options for statistically significant results
    stat_sign_widget = widgets.ToggleButtons(
        options=[0.05, 0.01, 0.2, 1.0],
        description="Statistical Significance:",
        disabled=False,
        button_style="",  # 'success', 'info', 'warning', 'danger' or ''
        tooltips=["95% (Default)", "99%", "80%", "100%"],
    )

    # Create a radio button widget with options for mean diff ranges
    mean_diff_widget = widgets.ToggleButtons(
        options=[0.10, 0.15, 0.20, 0.25],
        description="Mean Diff (BPS):",
        disabled=False,
        button_style="",  # 'success', 'info', 'warning', 'danger' or ''
        tooltips=["25 BPS (Default)", "20 BPS", "15 BPS", "10 BPS"],
    )

    # Create a radio button widget with options for showing statistic results
    show_stats_widget = widgets.ToggleButtons(
        options=["Exclude", "Include"],
        description="Show Statistical Results:",
        disabled=False,
        button_style="",  # 'success', 'info', 'warning', 'danger' or ''
        tooltips=["Exclude the stats (Default)", "Include the stats"],
    )

    # Create a radio button widget with options for showing plots
    show_plot_widget = widgets.ToggleButtons(
        options=["None ", "Strip Chart ", "Boxplot ", "List of Loans "],
        description="Charts and List of Loans:",
        disabled=False,
        button_style="",  # 'success', 'info', 'warning', 'danger' or ''
        tooltips=["Exclude the plots (Default)", "Include the plots"],
        icons=["times", "area-chart", "bar-chart", "list"],
    )

    # Use the interact function to apply the selected filters to the code
    interact(
        apply_filters,
        analysis=analysis_widget,
        stat_sign=stat_sign_widget,
        mean_diff_range=mean_diff_widget,
        show_stats=show_stats_widget,
        show_plot=show_plot_widget,
    )


# Set the function to be called when a file is selected
fc.register_callback(on_file_selected)

# Display the FileChooser widget
display(fc)
fc.title = "<b>Select File (.csv) for Ethnicity, Age, Sex, MSA Analysis</b>"

# Display the output widget
display(output)

---